In [1]:
#load .env variables
import os
from dotenv import load_dotenv
load_dotenv()
PWD = os.getenv('PWD')
from flask import Flask, app, redirect, url_for, render_template
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import pandas as pd
import requests
import json
from urllib.parse import urlencode
from datetime import timedelta

In [2]:
app = Flask(__name__)
# attach SQL to flask
db_name = PWD+'\\database'+'\\RVNUSDT.db'#rvnusdt
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_name
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

# this variable, db, will be used for all SQLAlchemy commands
db = SQLAlchemy(app)


In [3]:
class RVNUSDT(db.Model): 
    __tablename__ = 'd1'
    id = db.Column(db.Integer, primary_key=True)
    # 1499040000000,      // Open time
    # "0.01634790",       // Open
    # "0.80000000",       // High
    # "0.01575800",       // Low
    # "0.01577100",       // Close
    # "148976.11427815",  // Volume
    # 1499644799999,      // Close time
    # "2434.19055334",    // Quote asset volume
    # 308,                // Number of trades
    # "1756.87402397",    // Taker buy base asset volume
    # "28.46694368",      // Taker buy quote asset volume
    # "17928899.62484339" // Ignore.
    date_created = db.Column(db.DateTime, )
    open_time = db.Column(db.String(255))
    open_ = db.Column(db.String(255))
    high_ = db.Column(db.String(255))
    low_ = db.Column(db.String(255))
    close_ = db.Column(db.String(255))
    volume_ = db.Column(db.String(255))
    close_time = db.Column(db.String(255))
    quote_asset_volume = db.Column(db.String(255))
    number_of_trades = db.Column(db.String(255))
    taker_buy_base_asset_volume = db.Column(db.String(255))
    taker_buy_quote_asset_volume = db.Column(db.String(255))
    ignore_ = db.Column(db.String(255))

    # title_id = db.Column(db.Integer)
    # name_en = db.Column(db.String(255))
    # name_ru = db.Column(db.String(255))
    # max_ep = db.Column(db.Integer)
    # from_usr = db.Column(db.String(100))
    # to_usr = db.Column(db.String(100))

    def __init__(self, 
                        date_created ,
                        open_time ,
                        open_ ,
                        high_ ,
                        low_ ,
                        close_ ,
                        volume_ ,
                        close_time ,
                        quote_asset_volume ,
                        number_of_trades ,
                        taker_buy_base_asset_volume ,
                        taker_buy_quote_asset_volume ,
                        ignore_):
        self.date_created = date_created
        self.open_time = open_time
        self.open_ = open_
        self.high_ = high_
        self.low_ = low_
        self.close_ = close_
        self.volume_ = volume_
        self.close_time = close_time
        self.quote_asset_volume = quote_asset_volume
        self.number_of_trades = number_of_trades
        self.taker_buy_base_asset_volume = taker_buy_base_asset_volume
        self.taker_buy_quote_asset_volume = taker_buy_quote_asset_volume
        self.ignore_ = ignore_

In [4]:
db.create_all()

In [ ]:
# import requests
# import json
# from urllib.parse import urlencode

# pair = 'RVNUSDT'
# interval = '1m'
# limit = 1000
# startTime = 1569412800000
# endTime = 1569472800000
# endpoint = 'https://api.binance.com/api/v1/klines'
# data = urlencode(
#     {
#         'symbol': pair,
#         'interval': interval,
#         'limit': limit,
#         'startTime': startTime,
#         'endTime': endTime,
#     })
# resource = requests.get(url=f'{endpoint}?{data}', )
# data = json.loads(resource.text)
# data

In [5]:
class IterData(object): 
    '''docstring for IterData'''
    def __init__(self, init_time: datetime, interval: str, pair: str='RVNUSDT', limit: int=1000):
        super(IterData, self).__init__()
        self.pair = pair
        self.init_time = init_time
        self.start_time = init_time
        self.end_time = self.start_time + timedelta(minutes=limit)
        self.interval = interval
        self.limit = limit


    def calculate_interval(self, ): 
        # calculate interval for next request from API
        # if self.start_time != self.init_time:
        self.start_time = self.start_time + timedelta(minutes=self.limit)
        self.end_time = self.start_time + timedelta(minutes=self.limit)
            

    def iter_data_get(self, ): 
        dt_to_unix = lambda x: int(x.timestamp()*1000)
        # pair = 'RVNUSDT'
        # interval = '1m'
        # limit = 1000
        # startTime = 1569412800000
        # endTime = 1569472800000
        endpoint = 'https://api.binance.com/api/v1/klines'
        data = urlencode(
            {
                'symbol': self.pair,
                'interval': self.interval,
                'limit': self.limit,
                'startTime': dt_to_unix(self.start_time),
                'endTime': dt_to_unix(self.end_time),
            })
        r = requests.get(url=f'{endpoint}?{data}', )
        self.calculate_interval()
        return json.loads(r.text)


In [10]:
# COLLECT DATA TO DATABASE
from datetime import datetime
from time import sleep
d = IterData(
    # init_time=datetime(year=2019, month=9, day=25, hour=15)
    init_time=datetime(year=2019, month=9, day=25, hour=15),
    interval='1d'
)
# print(d.end_time > datetime(year=2019, month=12, day=1, hour=0, minute=0))
try:
    # while not d.end_time > datetime(year=2019, month=10, day=3, hour=0, minute=0):
    while not d.end_time > datetime(year=2022, month=1, day=1, hour=0, minute=0):
        sleep(1)
        print(d.end_time )
        data = d.iter_data_get()
        for i in data:

            record = RVNUSDT(
                        date_created = datetime.fromtimestamp(i[0]/1000) ,
                        open_time = str(i[0]) ,
                        open_ = str(i[1]) ,
                        high_ = str(i[2]) ,
                        low_ = str(i[3]) ,
                        close_ = str(i[4]) ,
                        volume_ = str(i[5]) ,
                        close_time = str(i[6]) ,
                        quote_asset_volume = str(i[7]) ,
                        number_of_trades = str(i[8]) ,
                        taker_buy_base_asset_volume = str(i[9]) ,
                        taker_buy_quote_asset_volume = str(i[10]) ,
                        ignore_ = str(i[11])
                        )
            db.session.add(record)
finally:
    db.session.commit()

2019-09-26 07:40:00
2019-09-27 00:20:00
2019-09-27 17:00:00
2019-09-28 09:40:00
2019-09-29 02:20:00
2019-09-29 19:00:00
2019-09-30 11:40:00
2019-10-01 04:20:00
2019-10-01 21:00:00
2019-10-02 13:40:00
2019-10-03 06:20:00
2019-10-03 23:00:00
2019-10-04 15:40:00
2019-10-05 08:20:00
2019-10-06 01:00:00
2019-10-06 17:40:00
2019-10-07 10:20:00
2019-10-08 03:00:00
2019-10-08 19:40:00
2019-10-09 12:20:00
2019-10-10 05:00:00
2019-10-10 21:40:00
2019-10-11 14:20:00
2019-10-12 07:00:00
2019-10-12 23:40:00
2019-10-13 16:20:00
2019-10-14 09:00:00
2019-10-15 01:40:00
2019-10-15 18:20:00
2019-10-16 11:00:00
2019-10-17 03:40:00
2019-10-17 20:20:00
2019-10-18 13:00:00
2019-10-19 05:40:00
2019-10-19 22:20:00
2019-10-20 15:00:00
2019-10-21 07:40:00
2019-10-22 00:20:00
2019-10-22 17:00:00
2019-10-23 09:40:00
2019-10-24 02:20:00
2019-10-24 19:00:00
2019-10-25 11:40:00
2019-10-26 04:20:00
2019-10-26 21:00:00
2019-10-27 13:40:00
2019-10-28 06:20:00
2019-10-28 23:00:00
2019-10-29 15:40:00
2019-10-30 08:20:00


In [12]:
df = pd.read_sql(
    'SELECT * FROM d1 ', #WHERE date_created > "2021-01-01 00:00:00.000000"
    # 'SELECT * FROM minute_scale WHERE id > 990 AND id < 1010',
    'sqlite:///' + db_name, 
    index_col='id',
    )
df
# df['date_created'][990:1010]

,date_created,open_time,open_,high_,low_,close_,volume_,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore_
id,,,,,,,,,,,,,
1,2019-09-26 03:00:00.000000,1569456000000,0.02650000,0.02900000,0.02474000,0.02799000,5815542.70000000,1569542399999,155295.89099100,1123,2094912.40000000,55573.03115800,0
2,2019-09-27 03:00:00.000000,1569542400000,0.02800000,0.02908000,0.02598000,0.02894000,5022722.60000000,1569628799999,138508.03157900,1246,2050297.60000000,56461.62048900,0
3,2019-09-28 03:00:00.000000,1569628800000,0.02894000,0.02949000,0.02768000,0.02817000,2577296.80000000,1569715199999,73180.62260500,673,892614.50000000,25478.76429100,0
4,2019-09-29 03:00:00.000000,1569715200000,0.02817000,0.03236000,0.02762000,0.03065000,26791880.40000000,1569801599999,820452.20132700,3331,9903083.30000000,303089.14684200,0
5,2019-09-30 03:00:00.000000,1569801600000,0.03083000,0.03138000,0.02763000,0.03069000,10140294.90000000,1569887999999,299951.48613500,2095,3834991.20000000,113853.71390600,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,2021-12-27 03:00:00.000000,1640563200000,0.09824000,0.11750000,0.09723000,0.11148000,465801268.30000000,1640649599999,51171195.18097900,181174,234039660.30000000,25722723.93920400,0
858,2021-12-28 03:00:00.000000,1640649600000,0.11143000,0.11454000,0.09421000,0.09547000,342088937.30000000,1640735999999,35457715.12473400,142769,168752477.80000000,17502461.74517300,0
859,2021-12-29 03:00:00.000000,1640736000000,0.09543000,0.09934000,0.08929000,0.09306000,188368307.20000000,1640822399999,17745218.31849900,71424,93441064.90000000,8809402.69921500,0


In [8]:
# delete all data from the table
db.session.query(RVNUSDT).delete() 
db.session.commit()

In [ ]:
datetime.today()

In [ ]:
pd.read_sql('SELECT * FROM _ WHERE "column" LIKE "query"',
            'sqlite:///' + db_name, 
            # index_col='id',
            )